<a href="https://colab.research.google.com/github/AnshulH/NLP-DL-Group2/blob/lstm/NLP_Project_1_Bi-LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -r /content/NLP-DL-Group2/
!cd /content && git clone https://github.com/AnshulH/NLP-DL-Group2

rm: cannot remove '/content/NLP-DL-Group2/': No such file or directory
Cloning into 'NLP-DL-Group2'...
remote: Enumerating objects: 105, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 105 (delta 26), reused 18 (delta 5), pack-reused 46
Receiving objects: 100% (105/105), 30.84 MiB | 15.62 MiB/s, done.
Resolving deltas: 100% (45/45), done.
Updating files: 100% (25/25), done.


In [2]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import words # added for detecting English words
import string
import nltk
import calendar
import re

nltk.download('stopwords')
nltk.download('words')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
def decode(vocab,corpus):
    
    text = ''
    for i in range(len(corpus)):
        wID = corpus[i]
        text = text + vocab[wID] + ' '
    return(text)

def encode(words,text):
    corpus = []
    tokens = text.split(' ')
    for t in tokens:
        try:
            wID = words[t][0]
        except:
            wID = words['<unk>'][0]
        corpus.append(wID)
    return(corpus)

def read_encode(file_name,vocab,words,corpus,threshold):
    
    wID = len(vocab)
    
    if threshold > -1:
        with open(file_name,'rt') as f:
            for line in f:
                line = line.replace('\n','')
                tokens = line.split(' ')
                for t in tokens:
                    try:
                        elem = words[t]
                    except:
                        elem = [wID,0]
                        vocab.append(t)
                        wID = wID + 1
                    elem[1] = elem[1] + 1
                    words[t] = elem

        temp = words
        words = {}
        vocab = []
        wID = 0
        words['<unk>'] = [wID,100]
        wID = 1
        words['<pad>'] = [wID,100]
        wID = 2
        words['<num>'] = [wID,100]
        wID = 3
        words['< end_bio >'] = [wID,100]
        vocab.append('<unk>')
        vocab.append('<pad>')
        vocab.append('<num>')
        vocab.append('< end_bio >')
        for t in temp:
            if temp[t][1] >= threshold:
                vocab.append(t)
                wID = wID + 1
                words[t] = [wID,temp[t][1]]
            
                    
    with open(file_name,'rt') as f:
        for line in f:
            line = line.replace('\n','')
            tokens = line.split(' ')
            for t in tokens:
                try:
                    wID = words[t][0]
                except:
                    wID = words['<unk>'][0]
                corpus.append(wID)
                
    return [vocab,words,corpus]

def read_alltext(file_name,batch_size):
  '''
  tokenizes data, removes digits, converts to wIDs, and provides a tensor of batched inputs

  inputs: 
    file_name = a complete file path to a mixed dataset containing text and labels
    sequence_length = the length of the desired input array
  outputs:
    x = a tensor containing wIDs of the input
  '''
  encodings = read_encode(file_name,[],{},[],3)
  word_dict = encodings[1]
  wid = []

  with open(file_name) as input:
    all_text = input.readlines()

  for line in all_text[:2000]:
    line = line.strip()
    line = re.sub("[\d-]", "",line)
    line = [token for token in word_tokenize(line.lower())]
    for token in line:
      word_info = word_dict.get(token)
      if word_info is None:
        wid.append(0)
      else:
        wid.append(word_info[0])

  x = torch.tensor(np.asarray(wid))

  num_batches = x.shape[0] // batch_size 
  x = x[:num_batches * batch_size]
  x = x.view(batch_size, num_batches)  

  return x

def make_tensors(file_name,sequence_length):
  '''
  creates a list of tuples containing a list with the tokenized bio and a fake/real label

  inputs: 
    file_name = a complete file path to a mixed dataset containing text and labels
    sequence_length = the length of the desired input array
  outputs:
    x = a tensor containing wIDs of the desired sequence length for each bio
    y = a tensor containing the corresponding label for each input
  '''
  
  # stop = set(stopwords.words('english') + list(string.punctuation))


  with open(file_name) as input:
    all_text = input.readlines()

# if anybody wants to do some work on the tokenization section, 
# you should work with the split_bios array, since it contains actual words

# split_bios = [([bio1, tokens1],[REAL]),([bio2, tokens2],[FAKE])]
  split_bios = []
  line_text = []
  seq_status = 0

  # seq_status = 0 marks the start of the bio
  # seq_status = 1 is the main text of the bio
  # seq_status = 2 is the end of the bio
  for line in all_text:
    line = line.strip()
    if "< start_bio >" in line:
      seq_status = 0
      continue
    if "< end_bio >" in line:
      seq_status = 2
      line_text.append(line)
      continue
    if seq_status == 0:
      # bio_person = line.strip(" = ").lower().split()
      # stop.update(bio_person) # could remove important words if name is meaningful
      seq_status = 1
    if seq_status == 1:
      line = re.sub("[\d-]", "",line)
      line = [token for token in word_tokenize(line.lower())]
      # line = [token for token in word_tokenize(line.lower()) if token not in stop]
      if len(line) > 0:
        line_text.extend(line)
    if seq_status == 2 and line !="":
      # stop -= set(bio_person)
      # if line == '[FAKE]': #updated code so now fake / real is binary
      #   line = 0
      # elif line == '[REAL]':
      #   line = 1
      # else:
      #   continue 
      split_bios.append((line_text,line))
      line_text = []

  encodings = read_encode(file_name,[],{},[],3)
  word_dict = encodings[1]
  vocab_length = max(encodings[2])

  bio_array = []
  seq_len = sequence_length
  y = []

  for bio in split_bios:
    wid = []
    for token in bio[0]:
      word_info = word_dict.get(token)
      if word_info is None:
        wid.append(0)
      else:
        wid.append(word_info[0])
    if len(wid) > seq_len:
      wid = wid[:seq_len-2]
      wid.append(word_dict.get('< end_bio >')[0]) #makes last token "end bio" no matter what
      wid.append(word_dict.get(bio[1])[0])
    # elif len(wid) < seq_len:
    #   wid.append(word_dict.get(bio[1])[0])
    #   pad_size = seq_len - len(wid)
    #   wid.extend(np.ones(pad_size,dtype=int))
    bio_array.append(np.asarray(wid))
    y.append(word_dict.get(bio[1])[0])
    # y.append(bio[1])

  x = bio_array
  y = torch.tensor(y)

  return x,y

In [5]:
all_text = read_alltext('/content/NLP-DL-Group2/hw#1/mix.train.tok',300)

In [6]:
def get_batch(data, seq_len, num_batches, idx):
    src = data[:, idx:idx+seq_len]                   
    target = data[:, idx+1:idx+seq_len+1]             
    return src, target

In [4]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import math
import time
import numpy as np
import sys
import argparse
import os

device = "cpu"

class FFNN(nn.Module):
    def __init__(self, vocab, words,d_model, d_hidden, dropout):
        super().__init__() 
    
        self.vocab = vocab
        self.words = words
        self.vocab_size = len(self.vocab)
        self.d_model = d_model
        self.d_hidden = d_hidden
        self.dropout = dropout
        self.embeds = nn.Embedding(self.vocab_size,d_model)
#          {perform other initializations needed for the FFNN}

    def forward(self, src):
        embeds = self.dropout(self.embeds(src))
#          {add code to implement the FFNN}
        pass
        # return x
                
    def init_weights(self):
      pass
#          {perform initializations}
             
class LSTM(nn.Module):
    def __init__(self,vocab,words,d_model,d_hidden,n_layers,dropout_rate, tie_weights):
        super().__init__()
        
        self.vocab = vocab
        self.words = words
        self.vocab_size = len(self.vocab)
        self.n_layers = n_layers
        self.d_hidden = d_hidden
        self.d_model = d_model
        self.embeds = nn.Embedding(self.vocab_size,d_model)
#          {perform other initializations needed for the LSTM}
        self.lstm = nn.LSTM(2*d_model, d_hidden, n_layers, dropout=dropout_rate, bidirectional=True, batch_first=True)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(2*d_hidden, self.vocab_size)

        if tie_weights:
          assert d_model == d_hidden, 'cannot tie, check dims'
          self.embeds.weight = self.fc.weight
        self.init_weights()
        
    def forward(self,src,h):
        embeds = self.dropout(self.embeds(src)) 
        out, h = self.lstm(embeds, h)
        out = self.dropout(out)
        predict = self.fc(out)
        return predict, h
    
    def init_weights(self):
        emb_range = 0.1
        init_range = 1/math.sqrt(self.d_hidden)
        self.embeds.weight.data.uniform_(-emb_range, emb_range)
        self.fc.weight.data.uniform_(-init_range, init_range)
        self.fc.bias.data.zero_()
        for i in range(self.n_layers):
            self.lstm.all_weights[i][0] = torch.FloatTensor(self.d_model,
                    self.d_hidden).uniform_(-init_range, init_range) 
            self.lstm.all_weights[i][1] = torch.FloatTensor(self.d_hidden, 
                    self.d_hidden).uniform_(-init_range, init_range) 
        
    def init_hidden(self, batch_size, device):
        hidden = torch.zeros(2*self.n_layers, batch_size, self.d_hidden).to(device)
        cell = torch.zeros(2*self.n_layers, batch_size, self.d_hidden).to(device)
        return hidden, cell

    def detach_hidden(self, hidden):
      hidden, cell = hidden
      hidden = hidden.detach()
      cell = cell.detach()
      return hidden,cell
 
def main():
    
    parser = argparse.ArgumentParser()
    parser.add_argument('-d_model', type=int, default=100)
    parser.add_argument('-d_hidden', type=int, default=100)
    parser.add_argument('-n_layers', type=int, default=2)
    parser.add_argument('-batch_size', type=int, default=20)
    parser.add_argument('-seq_len', type=int, default=30)
    parser.add_argument('-printevery', type=int, default=5000)
    parser.add_argument('-window', type=int, default=3)
    parser.add_argument('-epochs', type=int, default=20)
    parser.add_argument('-lr', type=float, default=0.0001)
    parser.add_argument('-dropout', type=int, default=0.35)
    parser.add_argument('-clip', type=int, default=2.0)
    parser.add_argument('-model', type=str,default='LSTM')
    parser.add_argument('-savename', type=str,default='lstm')
    parser.add_argument('-loadname', type=str)
    parser.add_argument('-trainname', type=str,default='wiki.train.txt')
    parser.add_argument('-validname', type=str,default='wiki.valid.txt')
    parser.add_argument('-testname', type=str,default='wiki.test.txt')

    params = {
        'd_model': 512,
        'd_hidden': 512,
        'n_layers': 2,
        'batch_size': 20,
        'seq_len': 30,
        'printevery': 500,
        'window': 3,
        'epochs': 20,
        'lr': 0.0001,
        'dropout': 0.35,
        'clip': 2.0,
        'model': 'LSTM',
        'savename': 'lstm',
        'loadname': None,
        'trainname': '/content/NLP-DL-Group2/hw#1/mix.train.tok',
        'validname': '/content/NLP-DL-Group2/hw#1/mix.valid.tok',
        'testname': '/content/NLP-DL-Group2/hw#1/mix.test.tok'
    }
    parser.add_argument("-f", required=False)
    
    # params = parser.parse_args()    
    # torch.manual_seed(0)
    
    [vocab,words,train] = read_encode(params['trainname'],[],{},[],3)
    print('vocab: %d train: %d' % (len(vocab),len(train)))
    [vocab,words,test] = read_encode(params['testname'],vocab,words,[],-1)
    print('vocab: %d test: %d' % (len(vocab),len(test)))
    params['vocab_size'] = len(vocab)

    train_loader = read_encode(params['trainname'],[],{},[],3)
    
    if params['model'] == 'FFNN':
      pass
#          {add code to instantiate the model, train for K epochs and save model to disk}
        
    if params['model'] == 'LSTM':
      pass

    if params['model'] == 'FFNN_CLASSIFY':
      pass
#          {add code to instantiate the model, recall model parameters and perform/learn classification}

    if params['model'] == 'LSTM_CLASSIFY':
      pass
#          {add code to instantiate the model, recall model parameters and perform/learn classification}
        
    print(params)
    
if __name__ == "__main__":
    main()

vocab: 35153 train: 3012820
vocab: 35153 test: 395279
{'d_model': 512, 'd_hidden': 512, 'n_layers': 2, 'batch_size': 20, 'seq_len': 30, 'printevery': 500, 'window': 3, 'epochs': 20, 'lr': 0.0001, 'dropout': 0.35, 'clip': 2.0, 'model': 'LSTM', 'savename': 'lstm', 'loadname': None, 'trainname': '/content/NLP-DL-Group2/hw#1/mix.train.tok', 'validname': '/content/NLP-DL-Group2/hw#1/mix.valid.tok', 'testname': '/content/NLP-DL-Group2/hw#1/mix.test.tok', 'vocab_size': 35153}


In [8]:
[vocab,words,train] = read_encode('/content/NLP-DL-Group2/hw#1/mix.train.tok',[],{},[],3)

rmv_list = []

for key,value in words.items():
  if value[1] < 10:
    rmv_list.append(value[0])

print(len(rmv_list))

updated_vocab = list(set(train) - set(rmv_list))

21431


In [ ]:
print(len(updated_vocab))
print(len(vocab))

In [11]:
[vocab,words,train] = read_encode('/content/NLP-DL-Group2/hw#1/mix.train.tok',[],{},[],3)

criterion = nn.CrossEntropyLoss()
data = all_text
batch_size = 20
seq_len = 30
clip = 2.0

model = LSTM(vocab,words,512,512,2,.35,True) 
optimizer = optim.Adam(model.parameters(), lr=.0008)

def LSTM_train(model, data, optimizer, criterion, batch_size, seq_len, clip, device):

  epoch_loss = 0
  model.train()

  num_batches = data.shape[-1]
  data = data[:, :num_batches - (num_batches -1) % seq_len]
  num_batches = data.shape[-1]

  hidden = model.init_hidden(batch_size, device)
    
  for idx in range(0, num_batches - 1, seq_len):  # The last batch can't be a src
      optimizer.zero_grad()
      hidden = model.detach_hidden(hidden)

      src, target = get_batch(data, seq_len, num_batches, idx)
      src, target = src.to(device), target.to(device)
      batch_size = src.shape[0]
      prediction, hidden = model(src, hidden)               

      prediction = prediction.reshape(batch_size * seq_len, -1)   
      target = target.reshape(-1)
      loss = criterion(prediction, target)
      
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
      optimizer.step()
      epoch_loss += loss.item() * seq_len
    
  return math.exp(epoch_loss / num_batches)

# print(LSTM_train(model,data,optimizer,criterion,batch_size,seq_len,clip,device))

In [12]:
def evaluate(model, data, criterion, batch_size, seq_len, device):

    epoch_loss = 0
    model.eval()
    num_batches = data.shape[-1]
    data = data[:, :num_batches - (num_batches -1) % seq_len]
    num_batches = data.shape[-1]

    hidden = model.init_hidden(batch_size, device)

    with torch.no_grad():
        for idx in range(0, num_batches - 1, seq_len):
            hidden = model.detach_hidden(hidden)
            src, target = get_batch(data, seq_len, num_batches, idx)
            src, target = src.to(device), target.to(device)
            batch_size= src.shape[0]

            prediction, hidden = model(src, hidden)
            prediction = prediction.reshape(batch_size * seq_len, -1)
            target = target.reshape(-1)

            loss = criterion(prediction, target)
            epoch_loss += loss.item() * seq_len
    return math.exp(epoch_loss / num_batches)

In [9]:
n_epochs = 50 
saved = True

train_data = read_alltext('/content/NLP-DL-Group2/hw#1/mix.train.tok',20)
valid_data = read_alltext('/content/NLP-DL-Group2/hw#1/mix.valid.tok',20)
test_data = read_alltext('/content/NLP-DL-Group2/hw#1/mix.test.tok',20)

lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=0)



In [ ]:
if False:
    model.load_state_dict(torch.load('best-val-lstm_lm.pt',  map_location=device))
    test_loss = evaluate(model, test_data, criterion, batch_size, seq_len, device)
    print(f'Test Perplexity: {test_loss:.3f}')
else:
    best_valid_loss = float('inf')

    for epoch in range(n_epochs):
        train_loss = LSTM_train(model, train_data, optimizer, criterion, 
                    20, seq_len, clip, device)
        valid_loss = evaluate(model, valid_data, criterion, 20, 
                    seq_len, device)
        
        lr_scheduler.step(valid_loss)

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), 'best-val-lstm_lm.pt')

        print(f'\tTrain Perplexity: {train_loss:.3f}')
        print(f'\tValid Perplexity: {valid_loss:.3f}')

In [ ]:
input = torch.tensor(word_tensors[0][0]).reshape(-1,1)
# print(input.reshape(-1,1))

def LSTM_Classify(input, model, vocab, device, seed=None):
  if seed is not None:
      torch.manual_seed(seed)
  model.eval()
  batch_size = len(input)
  hidden = model.init_hidden(batch_size, device)
  with torch.no_grad():
      src = input
      prediction, hidden = model(src, hidden)
      probs = torch.softmax(prediction[:, -1] , dim=-1)  
      real_prediction = probs[-1][638]
      fake_prediction = probs[-1][125]
      if real_prediction > fake_prediction:
        prediction = 638
      else:
        prediction = 125

  return prediction

LSTM_Classify(input,model,vocab,device)

In [ ]:
print(len(word_tensors[1]))

correct = 0

for index in range(len(word_tensors[1])):
  item = torch.tensor(word_tensors[0][index]).reshape(-1,1)
  prediction = LSTM_Classify(item,model,vocab,device)
  actual = word_tensors[1][index]
  if prediction == actual:
    correct += 1
  
print(correct/len(word_tensors[1]))

In [ ]:
model = LSTM(vocab,words,params['d_model'],params['d_hidden'],params['n_layers'],params['dropout']) 
optimizer = optim.Adam(model.parameters(), lr=params['lr'])
criterion = nn.CrossEntropyLoss()
data = word_tensors
y_data = torch.cat((word_tensors[0],word_tensors[1].reshape(-1,1)),1)[:params['seq_len'],1:513]
tie_weights = True
# hidden = model.init_weights(10)

In [ ]:
stop = set(stopwords.words('english') + list(string.punctuation))

In [ ]:
optimizer.zero_grad()
hidden = model.detach_hidden(hidden)

src = data[0][:10]
target = torch.cat((word_tensors[0],word_tensors[1].reshape(-1,1)),1)[1:513,:10]
src, target = src.to(device), target.to(device)
prediction, hidden = model(src, hidden)

prediction = prediction.reshape(-1)
target = target.reshape(-1)
print(prediction.size())
print(target.size())
loss = loss(prediction,target)

loss.backward()
optimizer.step()
epoch_loss += loss.item() * 10

print(epoch_loss)